# Space X Falcon 9 First Stage Landing Prediction

**Assignment: Machine Learning Prediction**

Space X advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars; other provides cost upward of165 million each, much of the savings is because Space X can reuse the first stage. Therefore if we can determine if the first stage will land, we can determine the cost of a launch. This information can be used if an alternate company wants to bid against space X for a rocket launch. In this lab, you will create a machine lwarning pipeline to predict if the first stage will land given the data from the preceding labs.